### Notebook 1.2: training on the main data of the competition (the Sartorius dataset)
We use the model pretrained in the notebook 1.1 and train it on the Sartorius dataset

In [1]:
import os
import detectron2
from pathlib import Path
import cv2
import numpy as np
import pycocotools.mask as mask_util
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
dataDir=Path('../')
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT='bitmask'
register_coco_instances('sartorius_train',{}, '../sartorius-annotations-coco-format/annotations_train.json', dataDir)
register_coco_instances('sartorius_val',{},'../sartorius-annotations-coco-format/annotations_val.json', dataDir)

[01/03 18:09:20 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json


In [3]:
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1
    false_positives = np.sum(matches, axis=0) == 0
    false_negatives = np.sum(matches, axis=1) == 0
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

In [6]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartorius_train",)
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = 'output_1.1/best_model.pth'
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.MAX_ITER = 10000
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
os.rename("output", "output_1.2")

[01/03 18:13:38 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[01/03 18:13:39 d2.data.datasets.coco]: Loaded 485 images in COCO format from ../sartorius-annotations-coco-format/annotations_train.json
[01/03 18:13:40 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (t

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (32, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (32,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (8, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in th

[01/03 18:13:41 d2.engine.train_loop]: Starting training from iteration 0
[01/03 18:13:52 d2.utils.events]:  eta: 0:51:41  iter: 19  total_loss: 3.084  loss_cls: 1.424  loss_box_reg: 0.4179  loss_mask: 0.6876  loss_rpn_cls: 0.3153  loss_rpn_loc: 0.2376  time: 0.4043  data_time: 0.2234  lr: 9.9905e-06  max_mem: 4652M
[01/03 18:14:03 d2.utils.events]:  eta: 0:52:31  iter: 39  total_loss: 3.08  loss_cls: 1.35  loss_box_reg: 0.4075  loss_mask: 0.6821  loss_rpn_cls: 0.2798  loss_rpn_loc: 0.2536  time: 0.4831  data_time: 0.2437  lr: 1.998e-05  max_mem: 4790M
[01/03 18:14:17 d2.utils.events]:  eta: 0:53:03  iter: 59  total_loss: 2.926  loss_cls: 1.216  loss_box_reg: 0.4607  loss_mask: 0.6761  loss_rpn_cls: 0.315  loss_rpn_loc: 0.2782  time: 0.5563  data_time: 0.3687  lr: 2.997e-05  max_mem: 5020M
[01/03 18:14:27 d2.utils.events]:  eta: 0:53:28  iter: 79  total_loss: 2.593  loss_cls: 1.019  loss_box_reg: 0.4638  loss_mask: 0.6466  loss_rpn_cls: 0.2256  loss_rpn_loc: 0.2291  time: 0.5390  data_

We choose the model obtained after iteration 6291.